In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Dropout, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
IMG_SIZE =48

train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,brightness_range=[0.5, 1.5],zoom_range=[0.9, 1.25] )

val_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,brightness_range= [0.5, 1.5],zoom_range=[0.9, 1.25])

train_generator = train_datagen.flow_from_directory(directory="smiledataset/train",target_size=(IMG_SIZE, IMG_SIZE),
        color_mode="grayscale",batch_size=16,class_mode="binary",shuffle=True,seed=42)

val_generator = val_datagen.flow_from_directory(directory="smiledataset/test",target_size=(IMG_SIZE, IMG_SIZE),
        color_mode="grayscale",batch_size=16,class_mode="binary",shuffle=True,seed=42)

Found 1468 images belonging to 2 classes.
Found 502 images belonging to 2 classes.


In [3]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size

#Intialize Neural Network'

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(IMG_SIZE,IMG_SIZE,1)))
model.add(MaxPooling2D())

model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())

model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(units=128, activation='relu'))

model.add(Dropout(0.25))

model.add(Dense(units=1, activation = 'sigmoid'))




In [ ]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("model10.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience =3,
                          verbose = 1,
                          restore_best_weights = True)

# put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# use a very small learning rate 
model.compile(loss='binary_crossentropy',optimizer = RMSprop(lr = 0.001),metrics = ['accuracy'])


model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=val_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=500)

model.save("model10.h5") 